# Generalizing code into a decorator

SPDX-License-Identifier: 0BSD

See also `decwrapornot.ipynb` for more general information about when decorators should define and return a wrapper function vs. modifying or using information from the function passed as its argument and passing that through.

In [1]:
import functools
import sys

## Example 1: `@peek_arg`

We begin with this function that squares its argument:

In [2]:
def square(number):
    """Find the square of a value."""
    return number**2

In [3]:
square(3)

9

Suppose we want—temporarily or permanently—to instrument it so it prints its argument when it is called:

In [4]:
def square(number):
    """Find the square of a value. Log the call."""
    print(f'square({number})')
    return number**2

In [5]:
square(3)

square(3)


9

If this is all we want to do—add instrumentation to a single function—then a decorator may be overkill. Just adding a line of code to `square` is sufficient. But if we want to make this change to multiple functions, or if the change were more involved and we wanted to separate its logic from that of the function it instruments, then it is reasonable to consider extracting it to a decorator.

We want to be able to put a decoration, `@peek_arg`, at the top of the definition of `square`, to cause `square` to have the added behavior of printing out how it was called. It would often be best to make `peek_arg` work on any function of any arity (and with or without keyword arguments), but for simplicity let’s assume we are only interested in using it to instrument functions that we call with a single positional argument and no keyword arguments.

Besides that, though, we don’t know anything about the behavior of functions that we will decorate with `@peek_arg`. More precisely, whether or not we know that, we don’t want to `@peek_arg` to rely on those details, since the whole point is to separate the logging functionality it performs from the behavior of the functions it will decorate.

Because the behavior of the function `peek_arg` returns adds to the behavior of the function passed to it:

- `peek_arg` will return a different function than the one passed to it, because the behavior of the function it returns will differ from the behavior of the function passed to it.

- The behavior of the function `peek_arg` will return *includes* the behavior of the function passed to it. So the function `peek_arg` returns will call the function passed to `peek_arg`.

To begin extracting the printing code to a decorator, we can make a function, `wrapper`, that works just like that:

In [6]:
def square(number):
    """Find the square of a value."""
    return number**2

In [7]:
def wrapper(number):
    """Find the square of a value. But first, report the function call."""
    print(f'square({number})')
    return square(number)

In [8]:
wrapper(3)

square(3)


9

Right now, `wrapper` depends on `square` beyond just referring to it by name, because it hard-codes the substring `square` in the interpolated string (the f-string) it prints. We can fix this by using `square.__name__` instead:

In [9]:
def wrapper(number):
    """Find the square of a value. But first, report the function call."""
    print(f'{square.__name__}({number})')
    return square(number)

In [10]:
wrapper(3)

square(3)


9

Now that `wrapper`’s only dependence on `square` is by the variable name `square`, we can create a factory function that takes this as a parameter and makes functions like `wrapper`. Doing this gives us:

In [11]:
def peek_arg(square):
    """Decorator wrapping a unary function and showing calls to it."""
    def wrapper(number):
        print(f'{square.__name__}({number})')
        return square(number)
    
    return wrapper

This no longer is specific to squaring, nor to functions whose argument is a number, so we can pick more general parameter/variable names:

In [12]:
def peek_arg(func):
    """Decorator wrapping a unary function and showing calls to it."""
    def wrapper(arg):
        print(f'{func.__name__}({arg})')
        return func(arg)
    
    return wrapper

We should also decorate `peek_arg` with `@functools.wraps(func)` so values of some important metadata attributes are copied from `func` to `wrapper`. This is desirable because `wrapper` will replace `func`—for example, it will be the function actually assigned to `square` when we decorate the definition of `square` with `@peek_arg`. Without doing this, anything that uses those metadata, such as doctest runners and the `help` builtin, will not work.

For more information on `functools.wraps`, see [its documentation](https://docs.python.org/3/library/functools.html#functools.wraps) and the **“Example 5: <code>@give_metadata_from(*wrapped*)</code>”** section below.

In [13]:
def peek_arg(func):
    """Decorator wrapping a unary function and showing calls to it."""
    @functools.wraps(func)
    def wrapper(arg):
        print(f'{func.__name__}({arg})')
        return func(arg)
    
    return wrapper

It works on `square`:

In [14]:
@peek_arg
def square(number):
    """Find the square of a value. Log the call."""
    return number**2

In [15]:
square(3)

square(3)


9

But just as well on other functions that are conceptually unrelated to `square`:

In [16]:
@peek_arg
def say_hi(name):
    """Say hi to someone, given their name. Log the call."""
    print(f'Hi, {name}!')

In [17]:
say_hi('Bob')

say_hi(Bob)
Hi, Bob!


Because we used `functools.wraps`, the metadata are available:

In [18]:
help(square)

Help on function square in module __main__:

square(number)
    Find the square of a value. Log the call.



In [19]:
help(say_hi)

Help on function say_hi in module __main__:

say_hi(name)
    Say hi to someone, given their name. Log the call.



## Example 2: `@call`

Sometimes we define a function just to have a local scope, and immediately call it. Other times, we will use the function later, but we still want to call it immediately. Both scenarios are most realistic in the setting of Jupyter notebooks.

We begin with this function that demonstrates how `_` is not assigned to when used as a `match`-`case` discard but is assigned to when used in tuple unpacking assignment:

In [20]:
def demonstrate_underscore():
    """Show the behavior of _ in various contexts."""
    match [10, 20]:
        case _, second:
            pass
    print(locals())

    _, second = [10, 20]
    print(locals())

demonstrate_underscore()

{'second': 20}
{'second': 20, '_': 10}


Already we can see that we don’t need a wrapper and shouldn’t use one: the functionality we want to augment the definition of `demonstrate_underscore` with is *already separate from that definition*. We’re not trying to change what happens when `demonstrate_underscore` is called—instead, we want to add behavior to *the act of defining it*.

We want to be able to decorate the definition with `@call` to produce the effect of the above code: defining the function, and calling it. Because we are not changing `demonstrate_underscore`’s behavior, we won’t return a different function, but instead just the same function.

Our `@call` decorator starts out as just an identity function—one that takes an argument, in this case a function, and returns it:

In [21]:
def call(demonstrate_underscore):
    # FIXME: This needs to do something.
    return demonstrate_underscore

In [22]:
@call
def demonstrate_underscore():
    """Show the behavior of _ in various contexts."""
    match [10, 20]:
        case _, second:
            pass
    print(locals())

    _, second = [10, 20]
    print(locals())

demonstrate_underscore()

{'second': 20}
{'second': 20, '_': 10}


Now we extract the call to `demonstrate_underscore` into the decorator:

In [23]:
def call(demonstrate_underscore):
    demonstrate_underscore()
    return demonstrate_underscore

And change the name of its parameter, since it is in no way specific to `demonstrate_underscore`, but instead works with any function that can be called with no arguments:

In [24]:
def call(func):
    func()
    return func

Now we can decorate the definition of `demonstrate_underscore` with `@call` instead of calling it after defining it:

In [25]:
@call
def demonstrate_underscore():
    """Show the behavior of _ in various contexts."""
    match [10, 20]:
        case _, y:
            pass
    print(locals())

    _, y = [11, 22]
    print(locals())

{'y': 20}
{'y': 22, '_': 11}


But `@call` also works with other unary functions that are not conceptually related to `demonstrate_underscore`:

In [26]:
@call
def hello():
    print('Hello, world!')

Hello, world!


## Example 3: `@thrice`

We begin by understating the glory and danger of bobcats:

In [27]:
def psa():
    """Print a bobcat reminder."""
    print('Watch out for bobcats!')

In [28]:
psa()

Watch out for bobcats!


Our caution is more likely to be heeded if we utter it thrice:

In [29]:
def psa():
    """Print a bobcat reminder, three times."""
    for _ in range(3):
        print('Watch out for bobcats!')

In [30]:
psa()

Watch out for bobcats!
Watch out for bobcats!
Watch out for bobcats!


There are many other amazing and ferocious beasts (giraffes, for example), so let’s extract this threefold repetition to a decorator that we can reuse.

The decorator will take a function like our original `psa`, and it will return a function with different behavior: what the function we put into the decorator does once, the function we get out of the decorator does three times. Since it will have different behavior—and since this behavior will be implemented by calling our function—this decorator will define and return a wrapper function.

The decorator won’t know what behavior it’s causing to happen three times in succession (it won’t necessarily be printing, even). This is how we know the function the decorator creates must *call* the function passed to the decorator—and thus that it will be a wrapper function.

To begin extracting the threefold repetition logic to the decorator, we can make a function, `wrapper`, that specifically calls `psa` three times:

In [31]:
def psa():
    """Print a bobcat reminder."""
    print('Watch out for bobcats!')

In [32]:
def wrapper():
    """Print three bobcat reminders."""
    for _ in range(3):
        psa()

In [33]:
wrapper()

Watch out for bobcats!
Watch out for bobcats!
Watch out for bobcats!


This situation is actually slightly simpler than what we had above in **“Example 1: `@peek_arg`”**, because `wrapper` already depends on `psa` only in that it refers to it by name—this `wrapper` doesn’t otherwise embody knowledge about what `psa` is or what it does.

So we can immediately create a factory function that takes `psa`—or any other function whose behavior we want to do three times—and makes functions like `wrapper`:

In [34]:
def thrice(psa):
    """Decorator wrapping a nullary function and calling it three times."""
    def wrapper():
        for _ in range(3):
            psa()
    
    return wrapper

This is no longer specific to issuing public service announcements about bobcats, so we can pick a more general parameter name.

In [35]:
def thrice(func):
    """Decorator wrapping a nullary function and calling it three times."""
    def wrapper():
        for _ in range(3):
            func()
    
    return wrapper

And decorate `wrapper` with `@functools.wraps(func)` so that metadata from `func` are appropriately propagated to `wrapper`:

In [36]:
def thrice(func):
    """Decorator wrapping a nullary function and calling it three times."""
    @functools.wraps(func)
    def wrapper():
        for _ in range(3):
            func()
    
    return wrapper

It works on `psa`:

In [37]:
@thrice
def psa():
    """Print three bobcat reminders."""
    print('Watch out for bobcats!')

In [38]:
psa()

Watch out for bobcats!
Watch out for bobcats!
Watch out for bobcats!


And on other unrelated functions:

In [39]:
@thrice
def sos():
    """Exclaim some information about our interpreter a few times."""
    print(f'A {sys.version_info.releaselevel} python is here now!')

In [40]:
sos()

A final python is here now!
A final python is here now!
A final python is here now!


## Example 4: <code>@repeat(*count*)</code>

A decorator like `thrice` can be further generalized. Why 3 times instead of some other number? For example, suppose we have:

In [41]:
def scotus():
    """Inspect the mind of a judge and report a thought."""
    print('I am the law!')

In [42]:
scotus()

I am the law!


We could do this, of course, but it’s far from scalable:

In [43]:
@thrice
@thrice
def scotus():
    """Inspect the minds on the US Supreme Court and report thoughts."""
    print('I am the law!')

In [44]:
scotus()

I am the law!
I am the law!
I am the law!
I am the law!
I am the law!
I am the law!
I am the law!
I am the law!
I am the law!


Instead, we can make a parameterized decorator—a higher-order function that returns an actual decorator—and decorate functions we want to repeat with a function-call expression. We will be able to use `@repeat(3)` for `psa` and `sos`, and `@repeat(9)` for `scotus`.

A parameterized decorator is, in a sense, not really a decorator. It would be an error to decorate a function definition like `@repeat`, since `repeat` is not what we want to pass a function. Rather, `repeat` is going to be a factory that we pass an integer *`count`* and that returns a function. The function it returns will then be passed a function that does something once, and *it* will return a function that does that thing *`count`* times. What `repeat` does it to make functions like `thrice` that themselves can be (and are meant to be) used as decorators.

Let’s start with the `thrice` function we already made:

In [45]:
def thrice(func):
    """Decorator wrapping a nullary function and calling it three times."""
    @functools.wraps(func)
    def wrapper():
        for _ in range(3):
            func()
    
    return wrapper

But change it to repeat *`count`* times instead of 3:

In [46]:
def repeat_count_times(func):
    """Decorator wrapping a nullary function and calling it count times."""
    @functools.wraps(func)
    def wrapper():
        for _ in range(count):
            func()
    
    return wrapper

But what is `count`?

In [47]:
@repeat_count_times
def scotus():
    """Inspect the minds on the US Supreme Court and report thoughts."""
    print('I am the law!')

In [48]:
try:
    scotus()
except NameError as error:
    print(error)

name 'count' is not defined


`count` is the parameter of our higher-order function `repeat`. So let’s put `repeat_count_times` into such a function:

In [49]:
def repeat(count):
    """Factory for decorators that repeat function some number of times."""
    def repeat_count_times(func):
        @functools.wraps(func)
        def wrapper():
            for _ in range(count):
                func()
        
        return wrapper
    
    return repeat_count_times

The naming used there is already fine, but since the most important thing to know about `repeat_count_times` when reading the code of `repeat` is that it—`repeat_count_times`—is the actual decorator, it is also reasonable to just name it `decorator`:

In [50]:
def repeat(count):
    """Factory for decorators that repeat function some number of times."""
    def decorator(func):
        @functools.wraps(func)
        def wrapper():
            for _ in range(count):
                func()
        
        return wrapper
    
    return decorator

Now we can call this with different values of `count` and apply the result as a decorator to various functions:

In [51]:
@repeat(9)
def scotus():
    """Inspect the minds on the US Supreme Court and report thoughts."""
    print('I am the law!')

In [52]:
scotus()

I am the law!
I am the law!
I am the law!
I am the law!
I am the law!
I am the law!
I am the law!
I am the law!
I am the law!


In [53]:
@repeat(3)
def psa():
    """Print three bobcat reminders."""
    print('Watch out for bobcats!')

In [54]:
psa()

Watch out for bobcats!
Watch out for bobcats!
Watch out for bobcats!


Expressions of the form <code>repeat(*count*)</code> are what *actually* decorate functions. We can even assign them to variables and use them that way:

In [55]:
twice = repeat(2)

In [56]:
@twice
def second_day():
    print('You received a turtle dove.')

In [57]:
second_day()

You received a turtle dove.
You received a turtle dove.


Note that, *outside* the implementation of `repeat`, *if* we name the a decorator obtained from `repeat`, we don’t call it `decorator`. Inside `repeat`’s implementation, the important thing is to be clear on exactly what function is the decorator. Outside it, the important thing is to know what the decorator we got from `repeat` actually does.

# Example 5: <code>@give_metadata_from(*wrapped*)</code>

Suppose we didn’t have `functools.wraps`—or, and this is the real motivation, suppose we want to understand how `functools.wraps` works by making something similar, a simplified version of it.

Let’s go back to our definition of `peek_arg`:

In [58]:
def peek_arg(func):
    """Decorator wrapping a unary function and showing calls to it."""
    @functools.wraps(func)
    def wrapper(arg):
        print(f'{func.__name__}({arg})')
        return func(arg)
    
    return wrapper

For this exercise, we’re not using the `functools.wraps` parameterized decorator from the Python standard library. Instead, we’ll build our own simpler but often adequate parameterized decorator, `give_metadata_from`, where decorating a wrapper function with `@give_metadata_from(wrapped)` will copy the appropriate metadata attributes from `wrapped` to the wrapper function.

As detailed in [the `functools.wrapped` documentation](https://docs.python.org/3.10/library/functools.html#functools.wraps), these attributes are those whose names are:

In [59]:
for name in functools.WRAPPER_ASSIGNMENTS:
    print(name)

__module__
__name__
__qualname__
__doc__
__annotations__


With the view of ultimately implementing and using `give_metadata_from`, we first change `peek_arg` to:

- no longer use `functools.wraps` but instead to show, commented out, the usage of `give_metadata_from` that we ultimately intend.

- perform the assignment to those five metadata attributes on `wrapper` before returning it.

In [60]:
def peek_arg(func):
    """Decorator wrapping a unary function and showing calls to it."""
    # @give_metadata_from(func)   <-- eventual usage, doesn't work yet
    def wrapper(arg):
        print(f'{func.__name__}({arg})')
        return func(arg)
    
    wrapper.__module__ = func.__module__
    wrapper.__name__ = func.__name__
    wrapper.__qualname__ = func.__qualname__
    wrapper.__doc__ = func.__doc__
    wrapper.__annotations__ = func.__annotations__
    
    return wrapper

We check that the code works:

In [61]:
@peek_arg
def fibonacci(n):
    """Slowly compute the Fibonacci number F(n) by simple recursion."""
    return n if n < 2 else fibonacci(n - 2) + fibonacci(n - 1)

In [62]:
fibonacci(3)

fibonacci(3)
fibonacci(1)
fibonacci(2)
fibonacci(0)
fibonacci(1)


2

In [63]:
help(fibonacci)  # Really, this only checks __name__ and __doc__.

Help on function fibonacci in module __main__:

fibonacci(arg)
    Slowly compute the Fibonacci number F(n) by simple recursion.



The process of going from the concrete to the abstract has two stages here:

1. First, we make a decorator `give_metadata_from_func`, that can operate on any `wrapper`.

2. After that, we will make a factory function (a parameterized decorator) that, when given `func`, returns a function that does the work of `give_metadata_from_func`.

Here’s `peek_arg` with commented-out usage of our intermediate abstraction `give_metadata_from_func` added:

In [64]:
def peek_arg(func):
    """Decorator wrapping a unary function and showing calls to it."""
    # @give_metadata_from(func)   <-- ultimate usage, doesn't work yet
    # @give_metadata_from_func    <-- intermediate usage, doesn't work yet either
    def wrapper(arg):
        print(f'{func.__name__}({arg})')
        return func(arg)
    
    wrapper.__module__ = func.__module__
    wrapper.__name__ = func.__name__
    wrapper.__qualname__ = func.__qualname__
    wrapper.__doc__ = func.__doc__
    wrapper.__annotations__ = func.__annotations__
    
    return wrapper

The `give_metadata_from_func` decorator will have `wrapper` as its one parameter—we’re applying it to `wrapper`, after all.

As with `@functools.wraps(func)`, decorating `wrapper` with `@give_metadata_from_func` will not wrap its parameter. Its parameter, `wrapper`, is already a wrapper function, but all `give_metadata_from_func` needs to do to it is assign to some of its attributes and return it. **We do not want to replace `wrapper` with a different function that delegates to it; instead, we want to make changes to `wrapper`’s attributes.**

In addition to depending on `wrapper` by taking it as an argument, `give_metadata_from_func` will also depends on `func`. So it will be a local function, taking `wrapper` as an argument and capturing `func`.

To avoid having to modify our assignments at this stage, and also because `wrapper` is a good name for this parameter, I’ll keep that name, even though, of course, the parameter could have any name. (In contrast, we cannot use a different name for `func`, at this point, in the implementation of `give_metadata_from_func`, since `func` is being captured from the enclosing scope.)

In [65]:
def peek_arg(func):
    """Decorator wrapping a unary function and showing calls to it."""
    # @give_metadata_from(func)   <-- ultimate usage, doesn't work yet
    # @give_metadata_from_func    <-- intermediate usage, doesn't work yet either
    def wrapper(arg):
        print(f'{func.__name__}({arg})')
        return func(arg)
    
    def give_metadata_from_func(wrapper):
        wrapper.__module__ = func.__module__
        wrapper.__name__ = func.__name__
        wrapper.__qualname__ = func.__qualname__
        wrapper.__doc__ = func.__doc__
        wrapper.__annotations__ = func.__annotations__
    
    give_metadata_from_func(wrapper)
    
    return wrapper

`give_metadata_from_func` doesn’t work as a decorator yet, but it’s close, and the above implementation of `peek_arg` does work.

In [66]:
@peek_arg
def fibonacci(n):
    """Slowly compute the Fibonacci number F(n) by simple recursion."""
    return n if n < 2 else fibonacci(n - 2) + fibonacci(n - 1)

In [67]:
fibonacci(3)

fibonacci(3)
fibonacci(1)
fibonacci(2)
fibonacci(0)
fibonacci(1)


2

The reason `give_metadata_from_func`, as defined above, wouldn’t work as a decorator, is that when you decorate a function definition, the function assigned to the name you use in the definition is the *return value* of the decorator. This facilitates decorators returning different functions from the ones they are passed, which is often useful—often we want to wrap—but which we don’t want here.

So we must make `give_metadata_from_func` return `wrapper`—to pass it through—rather than it current behavior if implicitly returning `None`:

In [68]:
def peek_arg(func):
    """Decorator wrapping a unary function and showing calls to it."""
    # @give_metadata_from(func)   <-- ultimate usage, doesn't work yet
    @give_metadata_from_func    # <-- almost works
    def wrapper(arg):
        print(f'{func.__name__}({arg})')
        return func(arg)
    
    def give_metadata_from_func(wrapper):
        wrapper.__module__ = func.__module__
        wrapper.__name__ = func.__name__
        wrapper.__qualname__ = func.__qualname__
        wrapper.__doc__ = func.__doc__
        wrapper.__annotations__ = func.__annotations__
        return wrapper
    
    give_metadata_from_func(wrapper)
    
    return wrapper

But there’s another change we need to make. It was convenient to put the definition of `give_metadata_from_func` below the definition of `wrapper`, since the contents of `give_metadata_from_func` were almost entirely written already, in that position. But `give_metadata_from_func` needs to be defined (so its name is assigned) prior to the first time we use it, and the first (and currently only) time we use it is to decorate the definition of `wrapper`.

In [69]:
try:
    @peek_arg
    def fibonacci(n):
        """Slowly compute the Fibonacci number F(n) by simple recursion."""
        return n if n < 2 else fibonacci(n - 2) + fibonacci(n - 1)
except UnboundLocalError as error:
    print(error)

cannot access local variable 'give_metadata_from_func' where it is not associated with a value


Fixing this makes it work:

In [70]:
def peek_arg(func):
    """Decorator wrapping a unary function and showing calls to it."""
    # @give_metadata_from(func)   <-- will replace @give_metadata_from_func

    def give_metadata_from_func(wrapper):
        wrapper.__module__ = func.__module__
        wrapper.__name__ = func.__name__
        wrapper.__qualname__ = func.__qualname__
        wrapper.__doc__ = func.__doc__
        wrapper.__annotations__ = func.__annotations__
        return wrapper

    @give_metadata_from_func
    def wrapper(arg):
        print(f'{func.__name__}({arg})')
        return func(arg)
    
    give_metadata_from_func(wrapper)
    
    return wrapper

In [71]:
@peek_arg
def fibonacci(n):
    """Slowly compute the Fibonacci number F(n) by simple recursion."""
    return n if n < 2 else fibonacci(n - 2) + fibonacci(n - 1)

In [72]:
fibonacci(3)

fibonacci(3)
fibonacci(1)
fibonacci(2)
fibonacci(0)
fibonacci(1)


2

We've created an abstraction, `give_metadata_from_func` that allows `wrapper` to vary. Now we need to create an abstraction, `give_metadata_from`, over that, to allow `func` to vary.